# Wildfire risk by US county

## Introduction

### Obtain all zipcodes for the state

In [158]:
state_name = 'CA'
filename = 'data/zipcodes_'+state_name+'.csv'
import pandas as pd
counties = pd.read_csv(filename, index_col=[0],names=['zipcode','city','county'])
print("%d zipcodes found..." % len(counties))
with pd.option_context('display.max_rows',10) : 
    print(counties)

2589 zipcodes found...
                     city       county
zipcode                               
90001         Los Angeles  Los Angeles
90002         Los Angeles  Los Angeles
90003         Los Angeles  Los Angeles
90004         Los Angeles  Los Angeles
90005         Los Angeles  Los Angeles
...                   ...          ...
96157    South Lake Tahoe    El Dorado
96158    South Lake Tahoe    El Dorado
96160             Truckee       Nevada
96161             Truckee       Nevada
96162             Truckee       Nevada

[2589 rows x 2 columns]


### Load cencus data 

In [159]:
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)
countydata = {}
skipped_zip=0
for zipcode in list(counties.index) : 
    data = search.by_zipcode(str(zipcode)).to_dict()
    try : 
        countydata[zipcode] = {
            "City":data["major_city"]+" "+data["state"],
            "Latitude":data["lat"],
            "Longitude":data["lng"],
            "Population":data["population"],
            "Density":int(data["population_density"]),
            "Occupancy":int(data["occupied_housing_units"] / data["housing_units"] * 100)/100,
            "Value":int(data["median_home_value"]/100)/10,
            "Income":int(data["median_household_income"]/100)/10
        }
    except:
        skipped_zip = skipped_zip+1
#         print("WARNING: error processing %s rbs, skipping" % (zipcode))
print("WARNING: missing data in %d out of %d zipcodes" % (skipped_zip,len(counties)))

### Load FEMA data

The data is retrieved from https://www.fema.gov/openfema-dataset-disaster-declarations-summaries-v2

In [160]:
fema_filename = 'data/DisasterDeclarationsSummaries.csv'
fire_data = pd.read_csv(fema_filename, index_col=[1],names=["femaDeclarationString",
                                                            "disasterNumber",
                                                            "state",
                                                            "declarationType",
                                                            "declarationDate",
                                                            "fyDeclared",
                                                            "incidentType",
                                                            "declarationTitle",
                                                            "ihProgramDeclared",
                                                            "iaProgramDeclared",
                                                            "paProgramDeclared",
                                                            "hmProgramDeclared",
                                                            "incidentBeginDate",
                                                            "incidentEndDate",
                                                            "disasterCloseoutDate",
                                                            "fipsStateCode",
                                                            "fipsCountyCode",
                                                            "placeCode",
                                                            "designatedArea",
                                                            "declarationRequestNumber",
                                                            "hash","lastRefresh"])
print('Types of incidents filed in the data set...')
print(set(fire_data['incidentType']))
# for item in list(fire_data.index) : 
#     print(item)
# with pd.option_context('display.max_rows',10) : 
#     print(fire_data)

Types of incidents filed in the data set...
{'Flood', 'Human Cause', 'Tsunami', 'Snow', 'Freezing', 'Severe Ice Storm', 'Dam/Levee Break', 'incidentType', 'Tornado', 'Chemical', 'Drought', 'Fire', 'Hurricane', 'Coastal Storm', 'Severe Storm(s)', 'Earthquake', 'Mud/Landslide', 'Fishing Losses', 'Volcano', 'Terrorist', 'Toxic Substances', 'Other', 'Typhoon'}


/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,5,8,9,10,11,15,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


*Assuming only fire incidents are relevent to this study

### Getting California fire incident data

In [161]:
import requests 
from bs4 import BeautifulSoup
import re
table_content = []
final_list = []
list_of_avail_years = range(2013,2019,1)
for yr in list_of_avail_years : 
    url = 'https://www.fire.ca.gov/incidents/'+str(yr)+'/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    html_content = {}
    for i in range(len(soup.find_all('div'))) : 
        try : 
            if soup.find_all('div')[i].attrs['class'] == ['section', 'incidents-esri-webmap']: 
                html_content = soup.find_all('div')[i].text
        except : 
            continue
    
    table_content=eval(html_content.partition('create_map([')[2].split("\n")[0][:-3])
    for t in table_content : 
        final_list.append(t)


In [162]:
fires_df = pd.DataFrame(final_list)

Example fire statistics on Ranch Fires

In [163]:
print(fires_df.loc[fires_df['Name'].str.contains('Ranch Fire')])

      IncidentId                              UniqueId  \
208          861  9487beeb-6c40-46fc-b4b3-a353a688a826   
323          976  a4e4603a-f0fc-4d7a-845b-6c668bfcb81e   
735         1388  5da65d7f-cc75-4c0c-b221-1a1e3000a120   
834         1487  2fd950a8-fadb-467c-ab59-88e7ea007738   
1253        1905  90b0daf7-5d84-42d9-bb35-bb78d4faf950   

                                Name  \
208                       Ranch Fire   
323                       Ranch Fire   
735                       Ranch Fire   
834                       Ranch Fire   
1253  Ranch Fire (Mendocino Complex)   

                                               Location   Latitude  \
208                       Southwest of Greenhorn Summit   0.000000   
323       off French Creek Rd, south of Shingle Springs   0.000000   
735      Off Grant Road, Mt. Hamilton, east of San Jose  37.334400   
834                            Ranch Camp Rd, Mt. Aukum  38.559707   
1253  off Highway 20 near Potter Valley, northeast o...  39.

Sorting by county by year 

In [164]:
county_list = []
county_list_repeated = set(fires_df['CountiesList'].tolist())
while ("" in county_list_repeated) : 
    county_list_repeated.remove("")
for item in county_list_repeated : 
    item_list = []
    item_list = item.split(",")
    if len(item_list) > 1 : 
        for c in item_list : 
            county_list.append(c.strip())
    else : 
        county_list.append(item)
county_set = set(county_list)
print('List of counties affect by fires : \n', county_set)

List of counties affect by fires : 
 {'Madera', 'El Dorado', 'Sacramento', 'Solano', 'Mariposa', 'Santa Cruz', 'Yolo', 'Mono', 'Yuba', 'Sonoma', 'Alpine', 'Alameda', 'San Diego', 'San Benito', 'Humboldt', 'Sierra', 'Colusa', 'Napa', 'Plumas', 'Santa Barbara', 'Riverside', 'Kings', 'Tulare', 'Los Angeles', 'Placer', 'Santa Clara', 'Amador', 'Fresno', 'Kern', 'Sutter', 'San Joaquin', 'Merced', 'Inyo', 'Tehama', 'Tuolumne', 'Lake', 'Del Norte', 'Siskiyou', 'Shasta', 'Monterey', 'Modoc', 'San Luis Obispo', 'Contra Costa', 'Nevada', 'Mendocino', 'Stanislaus', 'San Bernardino', 'Butte', 'Lassen', 'Trinity', 'San Mateo', 'Glenn', 'Calaveras', 'Orange', 'Marin', 'Ventura'}


In [165]:
print("Total of %d out 58 total counties in CA affected by fires" % len(county_set))

Total of 56 out 58 total counties in CA affected by fires


The county size information was acquired from https://www.counties.org/pod/square-mileage-county on Nov 15, 2019

In [166]:
county_size = pd.read_csv('data/counties_size.csv', names=['county','size'])
print("%d counties found..." % len(county_size))
with pd.option_context('display.max_rows',10) : 
    print(county_size)

58 counties found...
            county   size
0   San Bernardino  20057
1             Inyo  10181
2             Kern   8132
3        Riverside   7206
4         Siskiyou   6278
..             ...    ...
53          Amador    595
54           Marin    520
55       San Mateo    448
56      Santa Cruz    445
57   San Francisco     47

[58 rows x 2 columns]


Note, the area is quoted in square miles.

Checking that the county is listed in the manifest : 

In [167]:
county_manifest = county_size['county'].values.tolist()
for county in county_set : 
    if county not in county_manifest : 
        print("ERROR: Unknown County %s in fire data" % county)